In [ ]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from itertools import combinations
from scipy.stats import shapiro

In [ ]:
venta = pd.read_csv("venta_inmuebles.csv")

In [ ]:
df = venta.copy()


In [ ]:
df.info() 

In [ ]:
# Función para extraer las características de la columna y convertirlas en columnas separadas
def extraer_caracteristicas(fila):
    # Elimina los caracteres adicionales para convertir la cadena en una lista de tuplas
    fila = fila.strip("[]")  # Elimina los corchetes externos
    elementos = fila.split("), (")  # Divide la cadena en tuplas individuales
    
    datos_extraidos = {}
    for elemento in elementos:
        # Divide cada elemento en clave y valor
        clave, valor = elemento.split(", ", 1)
        clave = clave.replace("(", "").replace("'", "").strip()  # Limpia clave
        valor = valor.replace(")", "").replace("'", "").strip()  # Limpia valor
        datos_extraidos[clave.strip(':')] = valor  # Remueve ':' del final de la clave si está presente
    return datos_extraidos

# Aplicar la función a la columna 'Características' y expandir en columnas separadas
caracteristicas_df = df['Características'].dropna().apply(extraer_caracteristicas)
caracteristicas_expandidas = pd.json_normalize(caracteristicas_df)

# Combinar las columnas expandidas con el DataFrame original (elimina la columna 'Características' original)
df_limpio = pd.concat([df.drop(columns=['Características']), caracteristicas_expandidas], axis=1)

# Guardar el DataFrame limpio en un nuevo archivo CSV si deseas conservarlo
df_limpio.to_csv('venta_inmuebles_limpio.csv', index=False)

# Visualización del resultado
print(df_limpio.head())


In [ ]:
df_limpio

In [ ]:
# Lista de tipos de casa que queremos identificar
tipos_casa = ["Piso", "Chalet", "Adosado", "Casa", "Estudio", "Dúplex"]

# Función para extraer el tipo de casa de la descripción
def extraer_tipo_casa(descripcion):
    for tipo in tipos_casa:
        if tipo.lower() in descripcion.lower():
            return tipo
    return "Otro"  # Valor por defecto si no coincide con ninguno de los tipos en la lista

# Crear una nueva columna en el DataFrame con el tipo de casa
df_limpio['Tipo de Casa'] = df_limpio['Descripción'].apply(extraer_tipo_casa)

# Visualización del resultado
print(df_limpio[['Descripción', 'Tipo de Casa']].head())


In [ ]:
# Extraer la fecha de la columna 'Última Actualización' y convertirla a tipo datetime
df_limpio['Última Actualización'] = df_limpio['Última Actualización'].str.extract(r'(\d{2}/\d{2}/\d{4})')
df_limpio['Última Actualización'] = pd.to_datetime(df_limpio['Última Actualización'], format='%d/%m/%Y')

# Verificar el resultado
print(df_limpio['Última Actualización'].head())


0   2024-10-28
1   2024-10-28
2   2024-10-28
3   2024-07-03
4   2024-10-03
Name: Última Actualización, dtype: datetime64[ns]


In [ ]:
# Eliminar filas donde 'Precio' es NaN
df_limpio = df_limpio.dropna(subset=['Precio'])


In [ ]:
# Filtrar filas en 'Precio' que contienen caracteres no numéricos
valores_no_numericos = df_limpio[~df_limpio['Precio'].str.replace('.', '', regex=False).str.replace(',', '', regex=False).str.isnumeric()]

# Mostrar los resultados
print(valores_no_numericos[['Precio']])


In [ ]:
# Limpiar valores en la columna 'Precio', eliminando puntos y comas, y convertir a float
df_limpio.loc[:, 'Precio'] = df_limpio['Precio'].replace({r'\.': '', ',': ''}, regex=True).astype(float)


In [ ]:
# Rellenar valores nulos en columnas categóricas
df_limpio['Antigüedad'].fillna('No especificado', inplace=True)
df_limpio['Conservación'].fillna('No especificado', inplace=True)
df_limpio['Planta'].fillna('No especificado', inplace=True)

In [ ]:
 #Convertir 'Superficie construida' y 'Superficie útil' a numérico
df_limpio['Superficie construida'] = df_limpio['Superficie construida'].str.replace(' m²', '').astype(float)
df_limpio['Superficie útil'] = df_limpio['Superficie útil'].str.replace(' m²', '').astype(float)

In [ ]:
# Eliminar filas duplicadas
df_limpio.drop_duplicates(inplace=True)


In [ ]:
# Usar .title() para capitalizar la primera letra de cada palabra en los nombres de columna
df_limpio.columns = df_limpio.columns.str.title()

# Verificar los nombres de las columnas
print(df_limpio.columns)
